In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
filepath = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(filepath)

In [2]:
currencyFormat = '${:,.2f}'.format

## Player Count

* Display the total number of players


In [3]:
uniquePlayers = purchase_data['SN'].nunique()
print(f'{uniquePlayers} unique players.')


576 unique players.


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
#calculate values and move to DataFrame
summary_dict = {'Number of Unique Items': purchase_data['Item Name'].nunique(),
                'Average Price': purchase_data['Price'].mean(),
                'Number of Purchases': purchase_data['Purchase ID'].count(),
                'Total Revenue': purchase_data['Price'].sum()
               }
itemSummary_df = pd.DataFrame(summary_dict, index = [0])

#format and display DataFrame
itemSummary_df['Average Price'] = itemSummary_df['Average Price'].map(currencyFormat)
itemSummary_df['Total Revenue'] = itemSummary_df['Total Revenue'].map(currencyFormat)


itemSummary_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [5]:
# caluculate total unique players by gender with percent of total players
genderCounts_df = pd.DataFrame(purchase_data.groupby('Gender')['SN'].nunique())
genderCounts_df = genderCounts_df.rename(columns = {'SN': 'Total Count'})
genderCounts_df['Percentage of Players'] = genderCounts_df['Total Count'] / uniquePlayers

#Sort and Format DataFrame
genderCounts_df = genderCounts_df.sort_values('Total Count', ascending = False)
genderCounts_df = genderCounts_df.style.format({'Percentage of Players': "{:.2%}"})
genderCounts_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


# Observation

The demographics appear that males are the main audience for this gaming sample study with 84% ofthe total.


## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [6]:
#calculate each column by gender
purchaseByGender_df = pd.DataFrame(purchase_data.groupby('Gender')['Purchase ID'].count())
purchaseByGender_df['Average Purchase Price'] = purchase_data.groupby('Gender')['Price'].mean()
purchaseByGender_df['Total Purchase Value'] = purchase_data.groupby('Gender')['Price'].sum()
purchaseByGender_df['Avg Total Purchase per Person'] = purchaseByGender_df['Total Purchase Value'].divide(purchase_data.groupby('Gender')['SN'].nunique()) 

#format and rename results
purchaseByGender_df = purchaseByGender_df.rename(columns = {'Purchase ID': 'Purchase Count'})
purchaseByGender_df['Average Purchase Price'] = purchaseByGender_df['Average Purchase Price'].map(currencyFormat)
purchaseByGender_df['Total Purchase Value'] = purchaseByGender_df['Total Purchase Value'].map(currencyFormat)
purchaseByGender_df['Avg Total Purchase per Person'] = purchaseByGender_df['Avg Total Purchase per Person'].map(currencyFormat)

#display dataframe
purchaseByGender_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


# Observation

While males make up the larger audience, females tend to outspend their male counterparts by 10% or $0.40 per person on average.

## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [7]:
bins = [0, 9,14,19,24,29,34,39,999]
labels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

#get purchase data info and eliminate duplicate screennames
df = purchase_data[['SN', 'Age']]
df = df.drop_duplicates()

#bin by Age Counts and summarize
df['Total Count'] = pd.cut(df['Age'], bins = bins, labels = labels)
summaryAgeBin_df = pd.DataFrame(df['Total Count'].value_counts())

#calculate the Percent of total and format percentages
summaryAgeBin_df['Percentage of Players'] = summaryAgeBin_df['Total Count'] / len(df)

#sort and format DataFrame
summaryAgeBin_df = summaryAgeBin_df.sort_index()
summaryAgeBin_df = summaryAgeBin_df.style.format({'Percentage of Players': "{:.2%}"})

summaryAgeBin_df

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [8]:
#import Price data from purchase_data
df2 = purchase_data[['SN', 'Age', 'Price' ]]

#copy avoids chained indexing warning and prevents creating a view of the original dataframe.
df2 = df2.copy() 

#bin by Age Counts
df2['Price Bin'] = pd.cut(df2['Age'], bins = bins, labels = labels)


#first sum by unique user, then take average; some DataFrame cleanup
AvgTotPerPerson = df2.groupby(['SN', 'Price Bin'])['Price'].sum()
AvgTotPerPerson = AvgTotPerPerson.dropna()
AvgTotPerPerson = AvgTotPerPerson.reset_index()
AvgTotPerPerson = AvgTotPerPerson.drop('SN', 1)

#Create Summary DataFrame
summary_df = pd.DataFrame([df2.groupby(['Price Bin'])['Price'].sum()])
summary_df = pd.DataFrame.transpose(summary_df)
summary_df = summary_df.rename(columns = {'Price': 'Total Purchase Value'})

summary_df['Purchase Count'] = df2.groupby(['Price Bin'])['Price'].count()
summary_df['Average Purchase Price'] = df2.groupby(['Price Bin'])['Price'].mean()
summary_df['Avg Total Purchase per Person'] = AvgTotPerPerson.groupby(['Price Bin'])['Price'].mean()

#Format Currency Values
summary_df['Avg Total Purchase per Person'] = summary_df['Avg Total Purchase per Person'].map(currencyFormat)
summary_df['Average Purchase Price'] = summary_df['Average Purchase Price'].map(currencyFormat)
summary_df['Total Purchase Value'] = summary_df['Total Purchase Value'].map(currencyFormat)
summary_df


,Total Purchase Value,Purchase Count,Average Purchase Price,Avg Total Purchase per Person
Price Bin,,,,
<10,$77.13,23,$3.35,$4.54
10-14,$82.78,28,$2.96,$3.76
15-19,$412.89,136,$3.04,$3.86
20-24,"$1,114.06",365,$3.05,$4.32
25-29,$293.00,101,$2.90,$3.81
30-34,$214.00,73,$2.93,$4.12
35-39,$147.67,41,$3.60,$4.76
40+,$38.24,13,$2.94,$3.19


# Observation
The target market is definitely points to age 20-24.  While the sample size can affect the average purchase per person, the incoming revenue is largely centered in that age group.

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
#Set values in Dataframe
nameGroup = purchase_data.groupby('SN')['Price']

counting_df = pd.DataFrame(nameGroup.count())

counting_df = counting_df.rename(columns = {'Price': 'Purchase Count'})
counting_df['Average Purchase Price'] = nameGroup.mean()
counting_df['Total Purchase Price'] =  nameGroup.sum()

#format and display dataframe
counting_df = counting_df.sort_values('Total Purchase Price', ascending = False)
counting_df['Average Purchase Price'] = counting_df['Average Purchase Price'].map(currencyFormat)
counting_df['Total Purchase Price'] = counting_df['Total Purchase Price'].map(currencyFormat)
counting_df

,Purchase Count,Average Purchase Price,Total Purchase Price
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10
...,...,...,...
Ililsasya43,1,$1.02,$1.02
Irilis75,1,$1.02,$1.02
Aidai61,1,$1.01,$1.01


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [10]:
#import grouped Item data from source file
item_df = purchase_data.groupby(['Item ID', 'Item Name']).count()

#format and eliminated unneeded data fields
item_df = item_df.drop(['SN', 'Gender', 'Age', 'Price'], axis=1)
item_df = item_df.rename(columns = {'Purchase ID': 'Purchase Count'})

#calculate Total and avg 
item_df['Total Purchase Value'] = purchase_data.groupby(['Item ID', 'Item Name'])['Price'].sum()
item_df['Average Weighted Price'] = item_df['Total Purchase Value'] / item_df['Purchase Count']


item_df.head()


,,Purchase Count,Total Purchase Value,Average Weighted Price
Item ID,Item Name,,,
0,Splinter,4,5.12,1.2800
1,Crucifer,4,11.77,2.9425
2,Verdict,6,14.88,2.4800
3,Phantomlight,6,14.94,2.4900
4,Bloodlord's Fetish,5,8.50,1.7000


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [11]:
#Sort and Format DataFrame
item_df= item_df.sort_values('Total Purchase Value', ascending = False)

item_df['Average Weighted Price'] = item_df['Average Weighted Price'].map(currencyFormat)
item_df['Total Purchase Value'] = item_df['Total Purchase Value'].map(currencyFormat)

item_df.head()

,,Purchase Count,Total Purchase Value,Average Weighted Price
Item ID,Item Name,,,
92,Final Critic,13,$59.99,$4.61
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76,$4.23
82,Nirvana,9,$44.10,$4.90
145,Fiery Glass Crusader,9,$41.22,$4.58
103,Singed Scalpel,8,$34.80,$4.35


# Observation
There are very few games that are the true revenue drivers both in weighted price and total purchase value.  The sample size seems to be large enough to focus potential marketing toward the winners to drive sales.